In [1]:
import numpy as np  
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/used-electronics-data/Test.csv
/kaggle/input/used-electronics-data/Sample_Submission.xlsx
/kaggle/input/used-electronics-data/Train.csv


In [2]:
train = pd.read_csv('/kaggle/input/used-electronics-data/Train.csv')
test = pd.read_csv('/kaggle/input/used-electronics-data/Test.csv')
sub = pd.read_excel('/kaggle/input/used-electronics-data/Sample_Submission.xlsx')

In [3]:
train.shape, test.shape, sub.shape

((2326, 7), (997, 6), (997, 1))

In [4]:
train.head(5)

,Brand,Model_Info,Additional_Description,Locality,City,State,Price
0,1,name0 name234 64gb space grey,1yesr old mobile number 999two905two99 bill c...,878,8,2,15000
1,1,phone 7 name42 name453 new condition box acce...,101004800 1010065900 7000,1081,4,0,18800
2,1,name0 x 256gb leess used good condition,1010010000 seperate screen guard 3 back cover...,495,11,4,50000
3,1,name0 6s plus 64 gb space grey,without 1010020100 id 1010010300 colour 10100...,287,10,7,16500
4,1,phone 7 sealed pack brand new factory outet p...,101008700 10100000 xs max 64 gb made 10100850...,342,4,0,26499


In [5]:
test.head(3)

,Brand,Model_Info,Additional_Description,Locality,City,State
0,1,name0 55s66s66s778xxsxsmax etc,good condition 11months old single scratch we...,570,11,4
1,1,slightly used excellent condition name0 5 sale,101008700 1010030600 1010034300 10100192200 1...,762,8,2
2,1,name0 sx ios12 top letast model bill call,1010017300 delivery,60,13,5


In [6]:
train.isnull().sum()

Brand                     0
Model_Info                0
Additional_Description    0
Locality                  0
City                      0
State                     0
Price                     0
dtype: int64

In [7]:
train.nunique()

Brand                        4
Model_Info                2037
Additional_Description    2094
Locality                   970
City                        16
State                        9
Price                      469
dtype: int64

In [8]:
#train = train[(train['Price'] > 2500) & (train['Price'] < 100000)]
#train = train[train['Price'] > 399]

In [9]:
df = train.append(test,ignore_index=True)
df.shape

(3323, 7)

In [10]:
import re

def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)
df['Model_Info'] = df['Model_Info'].apply(remove_non_ascii)
#df['Additional_Description'] = df['Additional_Description'].apply(remove_non_ascii)

from sklearn.feature_extraction.text import TfidfVectorizer
tf1 = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', 
                     ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1)
df_text1 = tf1.fit_transform(df['Model_Info'])
df_text1 = pd.DataFrame(data=df_text1.toarray(), columns=tf1.get_feature_names())

#tf2 = TfidfVectorizer()
#df_text2 = tf2.fit_transform(df['Additional_Description'])
#df_text2 = pd.DataFrame(data=df_text2.toarray(), columns=tf2.get_feature_names())

In [11]:
df = pd.concat([df, df_text1], axis=1) 
df.shape

(3323, 2104)

In [12]:
df['Brand_Count'] = df['Brand'].map(df['Brand'].value_counts())
df['Locality_Count'] = df['Locality'].map(df['Locality'].value_counts())
df['CityCount'] = df['City'].map(df['City'].value_counts())
df['State_Count'] = df['State'].map(df['State'].value_counts())

In [13]:
#for c in ['Brand', 'Locality', 'City', 'State']:
#    df[c] = df[c].astype('category')

In [14]:
df.drop(['Model_Info','Additional_Description'], axis=1, inplace=True)

In [15]:
#df = pd.get_dummies(df, columns=['Brand', 'Locality', 'City', 'State'], drop_first=True)

In [16]:
train_df = df[df['Price'].isnull()!=True]
test_df = df[df['Price'].isnull()==True]
test_df.drop(['Price'], axis=1, inplace=True)

In [17]:
train_df['Price'] = np.log1p(train_df['Price'])

In [18]:
X = train_df.drop(labels=['Price'], axis=1)
y = train_df['Price'].values

X.shape, y.shape

((2326, 2105), (2326,))

In [19]:
from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [20]:
Xtest = test_df

In [21]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

errxgb = []
y_pred_totxgb = []

fold = KFold(n_splits=15, shuffle=True, random_state=42)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    xgb = XGBRegressor(random_state=42)
    xgb.fit(X_train, y_train)

    y_pred_xgb = xgb.predict(X_test)
    print("RMSLE: ", sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_xgb))))

    errxgb.append(sqrt(mean_squared_log_error(np.exp(y_test), np.exp(y_pred_xgb))))
    p = xgb.predict(Xtest)
    y_pred_totxgb.append(p)

RMSLE:  0.4236913025242859
RMSLE:  0.37866094195281774
RMSLE:  0.47062021731501147
RMSLE:  0.5416507089490135
RMSLE:  0.34528874765840034
RMSLE:  0.4550032130928629
RMSLE:  0.541573058848213
RMSLE:  0.513033796961394
RMSLE:  0.35480665277082507
RMSLE:  0.5075070411359691
RMSLE:  0.4693110466519935
RMSLE:  0.4737207244714871
RMSLE:  0.4998175730858012
RMSLE:  0.49815296651501023
RMSLE:  0.41982397374571406


In [22]:
np.mean(errxgb,0)

0.45951079771192

In [23]:
final = np.exp(np.mean(y_pred_totxgb,0))

In [24]:
sub['Price'] = final

In [25]:
sub.head()

,Price
0,15371.526367
1,8923.139648
2,15941.295898
3,45664.277344
4,5476.894531


In [26]:
sub.to_excel('Output.xlsx', index=False)